<div style="float:right; width:100px; text-align: center; margin: 10px;">
<img src="https://crypto-lake.com/assets/img/lake.png" alt="Lake"/>
</div>

# Momentum indicator

Backtest of a simple momentum indicator on 1m candle data. The logic was proposed by @BeatzXBT on twitter.

We use [crypto-lake.com](https://crypto-lake.com/#data) sample/free market data, FTRB-USDT market on Ascendex.

Quick links:
- [edit this notebook online](https://mybinder.org/v2/gh/crypto-lake/analysis-sharing/main?filepath=momentum_indicator.ipynb) using Binder
- [follow our activity on twitter](https://twitter.com/intent/user?screen_name=crypto_lake_com)

In [ ]:
import datetime

import numpy as np
import pandas as pd
import cufflinks as cf
import plotly.express as px
import statsmodels.api as sm

import lakeapi

cf.go_offline()

# This runs on the paid data only at the moment
# lakeapi.use_sample_data(anonymous_access=True)

In [ ]:
# Parameters
symbol = 'ADA-USDT'
exchange = 'BINANCE'

# Free sample data contain subset of the below time period
start = datetime.datetime(2023, 8, 1)
end = datetime.datetime(2023, 9, 1)

## Data

In [ ]:
print('Loading candles')
candles = lakeapi.load_data(
    table = 'candles',
    start = start,
    end = end,
    symbols = [symbol],
    exchanges = [exchange],
    drop_partition_cols = True,
).sort_values('origin_time')

In [ ]:
# Minimize the data
df = candles[['origin_time', 'close']]

## Logic

In [ ]:
depths = np.asarray([200, 100, 50, 25, 10]) # in minutes
n = len(depths)
# depths *= 10 # try a longer time frame?

# Signal logic
for depth in depths:
	df[f'ema_{depth}'] = np.log(df['close'] / df['close'].ewm(span=depth).mean()) * 100
df['trend_val'] = df[[f'ema_{depth}' for depth in depths]].ewm(span=n, axis = 1).mean().sum(axis=1)
df['pure_sum'] = df[[f'ema_{depth}' for depth in depths]].sum(axis=1)


# Future return for evaluation
df['future_return'] = df['close'].pct_change(10).shift(-10) * 100
df = df.dropna()

df[::3600].head(20)

## Evaluation

In [ ]:
df.set_index('origin_time')[['close', 'trend_val']].iplot(secondary_y='trend_val')

In [ ]:
# df.sample(1000).iplot(kind='scatter', x='trend_val', y='future_return', mode='markers', size=3)
px.scatter(df.sample(10_000), x='trend_val', y='future_return', trendline='ols')

In [ ]:
# Optional: install statsmodels for the next cell to run
!pip install -q statsmodels

In [ ]:
# Numerical linreg fit statistics
mod = sm.OLS(df['trend_val'], df['future_return'])
res = mod.fit()
print(res.summary())

## Conclusion



It seems the trend value has a slight negative correlation with future trend on our data. I don't consider the correlation significant enough for this signal to be used in real trading.

I also tried different settings, mostly longer ewm spans, but the results were similar.

---

### For reference: the original logic

In [ ]:
# This seems to be equivalent to pandas Series.ewm(span=window).mean()
def ema(arr_in: np.ndarray, window: int) -> np.ndarray:
    """
    Hyper-fast EMA implementation
    """
    
    n = arr_in.shape[0]
    ewma = np.empty(n, dtype=float)
    alpha = 2 / float(window + 1)
    w = 1
    ewma_old = arr_in[0]
    ewma[0] = ewma_old

    for i in range(1, n):
        w += (1-alpha)**i
        ewma_old = ewma_old*(1-alpha) + arr_in[i]
        ewma[i] = ewma_old / w
        
    return ewma

# This is replicated in the cell below in a batch/more-efficient way
def trend_feature(klines: np.ndarray, lengths: np.ndarray) -> float:
    """
    Make sure lengths are fed in from longest to shortest
    """

    closes = klines[:, 4]
    curr_price = closes[-1]
    n = len(lengths)
    vals = np.empty(n, dtype=float)

    for i in range(n):
        length = lengths[i]
        ema_val = ema(closes[-length:], length)[-1]

        # Safety check
        if ema_val == 0:
            vals[i] = vals[i-1]   

        else:
            vals[i] = np.log(curr_price / ema_val) * 100

    trend_val = ema(vals, n)

    return np.sum(trend_val)
